In [34]:
from pathlib import Path
from libvoikko import Voikko
import json

library_dir = Path('/opt/homebrew/opt/libvoikko/lib')
Voikko.setLibrarySearchPath(str(library_dir))

voikko = Voikko('fi')

if voikko.spell('talo'):
    print('Word is correct')
else:
    print('Word is incorrect')

print('Suggestions:')
for suggestion in voikko.suggest('talo'):
    print(f'  - {suggestion}')

print('Analysis:')
for result in voikko.analyze('talossa'):
    print(result)

print('Tokens:')
for token in voikko.tokens('Tämä on testi.'):
    print(token.tokenText)

Word is correct
Suggestions:
  - talo
Analysis:
{'BASEFORM': 'talo', 'CLASS': 'nimisana', 'FSTOUTPUT': '[Ln][Ica][Xp]talo[X]talo[Sine][Ny]ssa', 'NUMBER': 'singular', 'POSSIBLE_GEOGRAPHICAL_NAME': 'true', 'SIJAMUOTO': 'sisaolento', 'STRUCTURE': '=ppppppp', 'WORDBASES': '+talo(talo)'}
Tokens:
Tämä
 
on
 
testi
.


In [35]:
print(voikko.analyze('talossa'))
print(voikko.analyze('ja'))
print(voikko.analyze('ensimmäinen'))
print(voikko.analyze('pieni'))
print(voikko.analyze('nopeasti'))

# Comprehensive Voikko POS (CLASS) Reference
# Test various Finnish words to extract all available part-of-speech codes

test_words = {
    # Nouns
    'talo': 'house',
    'kissa': 'cat',
    'Helsinki': 'city name',
    'Matti': 'male name',
    'Virtanen': 'surname',
    
    # Verbs
    'juoksee': 'runs',
    'syödä': 'to eat',
    'mennä': 'to go',
    
    # Adjectives
    'suuri': 'big',
    'pieni': 'small',
    'punainen': 'red',
    
    # Adverbs
    'nopeasti': 'quickly',
    'hitaasti': 'slowly',
    
    # Conjunctions & particles
    'ja': 'and',
    'tai': 'or',
    'että': 'that',
    'kuin': 'like',
    
    # Numerals
    'yksi': 'one',
    'kaksi': 'two',
    'kymmenen': 'ten',
    'ensimmäinen': 'first',
    
    # Pronouns
    'hän': 'he/she',
    'minä': 'I',
    'sinä': 'you',
    
    # Prepositions
    'yli': 'over',
    'alle': 'under',
    'kanssa': 'with',
    
    # Other
    'ai': 'oh',
    'no': 'well',
}

pos_codes = {}

for word, meaning in test_words.items():
    try:
        analyses = voikko.analyze(word)
        for analysis in analyses:
            pos = analysis.get('CLASS', 'UNKNOWN')
            if pos not in pos_codes:
                pos_codes[pos] = {'examples': [], 'meanings': []}
            if word not in pos_codes[pos]['examples']:
                pos_codes[pos]['examples'].append(word)
                pos_codes[pos]['meanings'].append(meaning)
    except:
        pass

print("\n" + "="*80)
print("COMPREHENSIVE VOIKKO PART-OF-SPEECH (CLASS) CODES")
print("="*80)

for pos in sorted(pos_codes.keys()):
    print(f"\n{pos:20} Examples: {', '.join(pos_codes[pos]['examples'][:4])}")

print("\n" + "="*80)
print(f"Total unique POS codes found: {len(pos_codes)}")
print("="*80)

[{'BASEFORM': 'talo', 'CLASS': 'nimisana', 'FSTOUTPUT': '[Ln][Ica][Xp]talo[X]talo[Sine][Ny]ssa', 'NUMBER': 'singular', 'POSSIBLE_GEOGRAPHICAL_NAME': 'true', 'SIJAMUOTO': 'sisaolento', 'STRUCTURE': '=ppppppp', 'WORDBASES': '+talo(talo)'}]
[{'BASEFORM': 'ja', 'CLASS': 'sidesana', 'FSTOUTPUT': '[Lc][Xp]ja[X]ja', 'STRUCTURE': '=pp', 'WORDBASES': '+ja(ja)'}]
[{'BASEFORM': 'ensimmäinen', 'CLASS': 'lukusana', 'FSTOUTPUT': '[Lu][Xp]ensimmäinen[X]ensimmäi[Sn][Ny]nen', 'NUMBER': 'singular', 'SIJAMUOTO': 'nimento', 'STRUCTURE': '=ppppppppppp', 'WORDBASES': '+ensimmäinen(ensimmäinen)'}]
[{'BASEFORM': 'pieni', 'CLASS': 'laatusana', 'COMPARISON': 'positive', 'FSTOUTPUT': '[Ll][Xp]pieni[X]pien[Sn][Ny]i', 'NUMBER': 'singular', 'SIJAMUOTO': 'nimento', 'STRUCTURE': '=ppppp', 'WORDBASES': '+pieni(pieni)'}, {'BASEFORM': 'pieniä', 'CLASS': 'teonsana', 'FSTOUTPUT': '[Lt][Xp]pieniä[X]pien[Tt][Ai][P3][Ny][Ef]i', 'MOOD': 'indicative', 'NEGATIVE': 'false', 'NUMBER': 'singular', 'PERSON': '3', 'STRUCTURE': '=ppp

In [40]:
def get_mapped_pos(analysis_list):
    """
    Analyzes the detailed output from Voikko to determine if a word is an
    adjective (ADJ) or an adverb (ADV).

    Args:
        analysis_list: The list of analysis dictionaries returned by voikko.analyze().

    Returns:
        A string 'ADJ', 'ADV', or 'OTHER' based on the mapping logic.
    """
    
    # --- Define our mapping rules ---
    standalone_adverb_classes = {"seikkasana"}
    derived_adverb_cases = {"kerrontosti", "keinonto"}
    adjective_classes = {"laatusana", "nimisana_laatusana"}
    standard_noun_cases = {
        "nimento",      # Nominative
        "osanto",       # Partitive
        "sisaolento",   # Inessive
        "omanto",       # Genitive
        "ulkoolento",   # Adessive
        "sisaantulento", # Illative
    }

    # --- Apply the logic by first finding all possibilities ---
    has_nominative_adjective = False
    has_adverb_form = False
    has_other_adjective = False

    for analysis in analysis_list:
        v_class = analysis.get('CLASS')
        v_sijamuoto = analysis.get('SIJAMUOTO')

        # Check for high-confidence adverbs
        if v_class in standalone_adverb_classes:
            has_adverb_form = True
        if v_class in adjective_classes and v_sijamuoto in derived_adverb_cases:
            has_adverb_form = True
        
        # Check for high-confidence adjectives (nominative case is a strong signal)
        if v_class in adjective_classes and v_sijamuoto == "nimento":
            has_nominative_adjective = True
        
        # Check for other standard adjectives
        elif v_class in adjective_classes and v_sijamuoto in standard_noun_cases:
            has_other_adjective = True

    # --- Decision Logic (Prioritized) ---
    # The nominative case is the base form and a very strong signal for an adjective.
    # We prioritize it to correctly identify words like 'nopein'.
    if has_nominative_adjective:
        return "ADJ"
    
    if has_adverb_form:
        return "ADV"
        
    if has_other_adjective:
        return "ADJ"

    return "OTHER"


def run_tests():
    """
    Initializes Voikko and runs a series of tests to validate the POS mapping logic.
    """
    print("Initializing Voikko for Finnish (fi)...")
    try:
        voikko = Voikko("fi")
    except Exception as e:
        print(f"Failed to initialize Voikko: {e}")
        print("Please ensure the Finnish dictionaries (especially 'morpho') are installed correctly.")
        return

    test_cases = [
        # --- Basic Adverbs (-sti) ---
        ("hitaasti", "ADV"),
        ("kauniisti", "ADV"),
        ("pahasti", "ADV"),
        ("oikein", "ADV"),

        # --- Basic Adjectives (various cases) ---
        ("hidas", "ADJ"),
        ("kaunista", "ADJ"),
        ("pahassa", "ADJ"),
        ("suuri", "ADJ"),
        ("suurta", "ADJ"),
        ("isolla", "ADJ"),

        # --- Advanced Adverbs (instructive case & standalone) ---
        ("hyvin", "ADV"),
        ("usein", "ADV"),
        ("paremmin", "ADV"),
        ("nopeimmin", "ADV"),

        # --- Advanced Adjectives (comparative/superlative) ---
        ("parempi", "ADJ"),
        ("nopein", "ADJ"),      # This one failed before

        # --- Negative Cases (should NOT be ADJ or ADV) ---
        ("talo", "OTHER"),
        ("juosta", "OTHER"),
        ("ja", "OTHER"),
        ("minä", "OTHER"),
    ]

    print("\n--- Running POS Logic Tests ---\n")
    all_passed = True

    for word, expected_pos in test_cases:
        analysis_result = voikko.analyze(word)
        actual_pos = get_mapped_pos(analysis_result)

        if actual_pos == expected_pos:
            print(f"✅ PASS: '{word}' -> Correctly identified as {actual_pos}")
        else:
            all_passed = False
            print(f"❌ FAIL: '{word}' -> Expected {expected_pos}, but got {actual_pos}")
            print(f"      Voikko Analysis: {analysis_result}")

    print("\n--- Test Summary ---")
    if all_passed:
        print("🎉 All tests passed successfully!")
    else:
        print("🔥 Some tests failed. Please review the output above.")

run_tests()


Initializing Voikko for Finnish (fi)...

--- Running POS Logic Tests ---

✅ PASS: 'hitaasti' -> Correctly identified as ADV
✅ PASS: 'kauniisti' -> Correctly identified as ADV
✅ PASS: 'pahasti' -> Correctly identified as ADV
❌ FAIL: 'oikein' -> Expected ADV, but got ADJ
      Voikko Analysis: [{'BASEFORM': 'oikea', 'CLASS': 'nimisana_laatusana', 'COMPARISON': 'positive', 'FSTOUTPUT': '[Lnl][Xp]oikea[X]oike[Sin][Nm]in', 'NUMBER': 'plural', 'SIJAMUOTO': 'keinonto', 'STRUCTURE': '=pppppp', 'WORDBASES': '+oikea(oikea)'}, {'BASEFORM': 'oikea', 'CLASS': 'laatusana', 'COMPARISON': 'superlative', 'FSTOUTPUT': '[Lnl][Xp]oikea[X]oike[Sn][Ny][Cs]in', 'NUMBER': 'singular', 'SIJAMUOTO': 'nimento', 'STRUCTURE': '=pppppp', 'WORDBASES': '+oikea(oikea)'}, {'BASEFORM': 'oikein', 'CLASS': 'seikkasana', 'FSTOUTPUT': '[Ls][Xp]oikein[X]oikein', 'STRUCTURE': '=pppppp', 'WORDBASES': '+oikein(oikein)'}]
✅ PASS: 'hidas' -> Correctly identified as ADJ
✅ PASS: 'kaunista' -> Correctly identified as ADJ
✅ PASS: 'pah

In [37]:
voikko.terminate()